※ 'prob10_1.csv'과 'prob10_2.csv'는 시간, 예약 시간, 그리고 예약자이름의 column으로 구성되어 있다.<br>
∙ csv 파일 2개를 glob을 활용해서 불러와라.(prob10_1.csv는 tel이라는 이름, prob10_2.csv는 internet이라는 이름으로 지정)<br>
∙ tel과 internet을 concat을 활용해 합치고 시간에 따라 정렬하여라(index는 위에서 순서대로 0~5까지 나오도록 해주기)<br>
  이 때 tel과 internet이라는 key를 지정해주기!!<br>
∙ '종료시간'이라는 column을 만들고 각 예약별 종료시간을 채워넣어라.(time_cal라는 함수를 정의해 활용)

In [7]:
import pandas as pd
from glob import glob

In [41]:
filenames = glob('prob10*.csv')
dataframes = [pd.read_csv(i, encoding='cp949') for i in filenames]
tel = dataframes[0]
internet = dataframes[1]
tel

,시간,예약 시간,예약자이름
0,1:00,1시간,은하
1,3:30,1시간,성웅
2,5:00,1시간,예성


In [42]:
add = pd.concat([tel,internet],keys=['tel','internet'],axis=0, ignore_index=True)
add = add.sort_values(by='시간')
add = add.reset_index(drop=True)
add

,시간,예약 시간,예약자이름
0,1:00,1시간,은하
1,2:00,1시간,펭수
2,3:00,1시간,근호
3,3:30,1시간,성웅
4,5:00,1시간,예성
5,6:00,1시간,뿡뿡이


In [43]:
def time_cal(row):
    return str(int(row['시간'][0])+int(row['예약 시간'][0]))+row['시간'][1:]
add['종료시간'] = add.apply(time_cal, axis=1)
add

,시간,예약 시간,예약자이름,종료시간
0,1:00,1시간,은하,2:00
1,2:00,1시간,펭수,3:00
2,3:00,1시간,근호,4:00
3,3:30,1시간,성웅,4:30
4,5:00,1시간,예성,6:00
5,6:00,1시간,뿡뿡이,7:00


In [1]:
import pandas as pd

In [3]:
#데이터 불러오기
entry = pd.read_csv('출전명단.csv', encoding='cp949', index_col='경기')
pitcher = pd.read_csv('투수자료.csv', encoding='cp949', index_col='이름')
batter = pd.read_csv('타자자료.csv', encoding='cp949', index_col='이름')

In [4]:
#데이터 살펴보기
print(entry.head())
print(pitcher.head())
print(batter.head())

    투수  타자
경기        
1   예성  은하
2   근호  성웅
3   성웅  예성
4   예성  예성
5   성웅  근호
      승률  수당
이름          
근호  0.48   7
성웅  0.50   8
예성  0.53   9
은하  0.43   5
      승률  수당
이름          
근호  0.54   9
성웅  0.43   5
예성  0.44   5
은하  0.47   6


In [5]:
# 아래와 같은 자료를 만든다.
merge = pd.merge(entry, pitcher, left_on='투수', right_on='이름', how='left')
merge = pd.merge(merge, batter, left_on='타자', right_on='이름', suffixes=('_투수', '_타자'), how='left')
merge

,투수,타자,승률_투수,수당_투수,승률_타자,수당_타자
0,예성,은하,0.53,9,0.47,6
1,근호,성웅,0.48,7,0.43,5
2,성웅,예성,0.50,8,0.44,5
3,예성,예성,0.53,9,0.44,5
4,성웅,근호,0.50,8,0.54,9
5,근호,은하,0.48,7,0.47,6
6,성웅,예성,0.50,8,0.44,5
7,은하,성웅,0.43,5,0.43,5
8,예성,은하,0.53,9,0.47,6
9,은하,근호,0.43,5,0.54,9


In [6]:
#승률, 수당 구하기
merge['승률'] = (merge['승률_투수'] + merge['승률_타자'])/2
merge['수당'] = merge['수당_투수'] + merge['수당_타자']
merge

,투수,타자,승률_투수,수당_투수,승률_타자,수당_타자,승률,수당
0,예성,은하,0.53,9,0.47,6,0.500,15
1,근호,성웅,0.48,7,0.43,5,0.455,12
2,성웅,예성,0.50,8,0.44,5,0.470,13
3,예성,예성,0.53,9,0.44,5,0.485,14
4,성웅,근호,0.50,8,0.54,9,0.520,17
5,근호,은하,0.48,7,0.47,6,0.475,13
6,성웅,예성,0.50,8,0.44,5,0.470,13
7,은하,성웅,0.43,5,0.43,5,0.430,10
8,예성,은하,0.53,9,0.47,6,0.500,15
9,은하,근호,0.43,5,0.54,9,0.485,14


In [7]:
#상관계수 구하기
merge[['승률', '수당']].corr()

,승률,수당
승률,1.000000,0.995097
수당,0.995097,1.000000


In [8]:
#1차적으로 승률이 0.48이 넘는 조합을 찾는다.
combi_candidate = merge[['투수', '타자', '수당']][merge['승률'] >= 0.48]
combi_candidate

,투수,타자,수당
0,예성,은하,15
3,예성,예성,14
4,성웅,근호,17
8,예성,은하,15
9,은하,근호,14


In [9]:
#이 중 수당이 가장 낮은 조합을 출력한다.
print("최적조합 :" + str(combi_candidate.sort_values('수당')[['투수', '타자']].head(1).values))

최적조합 :[['예성' '예성']]
